# Test-Charaktere
Für erste Tests erst einmal 2 simple Charaktere um die Grundlagen zu testen.

In [464]:
# Modul um Würfel zu simulieren
import random
random.seed(1321345589)

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd

In [465]:
def w20():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 20 zurück.
    """

    return random.randint(1, 20)


def w6():
    """
    Simple Simulation eines W6 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 6 zurück.
    """

    return random.randint(1, 6)

In [466]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_fertigkeiten = pd.read_excel("data/DSA5_Talente_nach_Kategorie.xlsx")
dsa5_fertigkeiten.dropna(inplace=True) # entfernt leere Zeilen

# Um den späteren Zugriff zu erleichtern wird die Spalte Eigenschaftsprobe geändert:
# Die Werte sind beim Einlesen ein String in der Form 'P1/P2/P3', dieser wird am Seperator
# '/' gesplittet. Hierbei wird eine Liste erzeugt mit den drei Elementen [P1, P2, P3].
dsa5_fertigkeiten["Eigenschaftsprobe"] = dsa5_fertigkeiten.Eigenschaftsprobe.apply(lambda x: x.split("/"))

print("Die ersten 10 Zeilen der Datentabelle der Fertigkeiten")
dsa5_fertigkeiten.head(10)

Die ersten 10 Zeilen der Datentabelle der Fertigkeiten


,Kategorie,Talent,Steigerungsfaktor,Eigenschaftsprobe
0,Körpertalente,Fliegen,C,"[MU, IN, GE]"
1,Körpertalente,Gaukeleien,A,"[MU, CH, FF]"
2,Körpertalente,Klettern,B,"[MU, GE, KK]"
3,Körpertalente,Körperbeherrschung,B,"[GE, GE, KO]"
4,Körpertalente,Kraftakt,B,"[KO, KK, KK]"
5,Körpertalente,Reiten,B,"[CH, GE, KK]"
6,Körpertalente,Schwimmen,B,"[GE, KO, KK]"
7,Körpertalente,Selbstbeherschung,D,"[MU, MU, KO]"
8,Körpertalente,Singen,A,"[KL, CH, KO]"
9,Körpertalente,Sinnesschärfe,D,"[KL, IN, IN]"


## Glossar

In [467]:
# Kurzes Glossar durch die ganzen Abkürzungen (besser für die Textausgabe)
# Benutzt ein dictionary, also glossar["MU"] ergibt als Ausgabe "Mut"
glossar = {
    "MU": "Mut",
    "KL": "Klugheit",
    "IN": "Intuition",
    "CH": "Charisma",
    "FF": "Fingerfertigkeit",
    "GE": "Gewandheit",
    "KO": "Konstitution",
    "KK": "Körperkraft",

    "LP": "Lebenspunkt",
    "AsP": "Astralpunkt",
    "AUS": "Ausdauer",
    "INIT": "Initiative",
    "AP": "Abenteurerpunkte"
}

# Das Glossar wird verdoppelt, wobei die Dopplung Keys und Values miteinander vertauscht.
# Also das dict glossar ist nach dem "glossar[key] = value"-Prinzip aufgebaut, dies wird
# mit der dict comprehension (also {val: key for ...}) gedreht. Das daraus entstehende
# neue dict wird dann dem "alten" Glossar beigefügr (der Operator | verbindet zwi dicts.
# Damit ist jetzt auch glossar["Mut"] = "MU" gewährleistet.
glossar = glossar | {val: key for key, val in glossar.items()}

# Liste der vorhandenen Eigenschaften welche jeder Charakter besitzt
# Wir nutzen eine Liste, damit über die Einträg iteriert werden kann
dsa3_eigenschaften = ["MU", "KL", "IN", "CH", "FF", "GE", "KO", "KK"]

# Dictionary der DSA3 Fertigkeiten. Hier wurde ein dict gewählt, welche als Key die Fertigkeit
# hat und als Value zuerst eine Liste mit den einzelnen Eigenschaften für den Check und danach ein
# Integer, welcher die Ausgleichspunkte darstellen soll.
dsa3_fertigkeiten = {
    "Tierkunde": [["KL", "KL", "IN"], w6()]
}

# Die Startwerte sind erst einmal für alle Charaktere gleich. Kann man später noch bearbeiten...
# Bspw. kann bei der Charaktererstellung AsP mit 4 multipiziert werden, wenn man einen Magier wählt.
# Auch hier wird in dict verwendet-
dsa3_startwerte = {
    "LP": 12,
    "AsP": 12,
    "AUS": 12,
    "INIT": 12,
    "AP": 0
}

In [468]:
def generate_eigenschaften(skills: list):
    """
    Generiert ein dictionary an random Werten (1-20) für die eigenschaften eines
    DSA Charakters. Diese dienen erst einmal nur Tetszwecken und können später
    noch verfeinert werden.

    : skills : Eine Liste
    """

    eigenschaften = {}

    for skill in skills:
        eigenschaften[skill] = w20()

    return eigenschaften


def generate_fertigkeiten(skills: dict):
    """
    
    """

    pass



def generate_charakterbogen(name: str, alter: int, geschlecht: str, rasse: str, heldentyp: str):
    """

    """

    label = ["name", "alter", "geschlecht", "rasse", "heldentyp"]
    infos = [name, alter, geschlecht, rasse, heldentyp]

    charakterbogen = dict(zip(label, infos)) | dsa3_startwerte
    
    charakterbogen["eigenschaften"] = generate_eigenschaften(dsa3_eigenschaften)
    charakterbogen["fertigkeiten"] = dsa3_fertigkeiten

    return charakterbogen


In [469]:
data_parnileo_parnilei = generate_charakterbogen("Parnileo Parnilei", 461, "männlich", "Mensch", "Magier")
data_edgar_allan_parn = generate_charakterbogen("Edgar Allan Parn", 216, "männlich", "Mensch", "Magier")

In [470]:
class Einheit():
    def __init__(self, file):
        if file is not None:
            for key, value in file.items():
                setattr(self, key, value)


    def eigenschaftsprobe(self, probe: str, bias: int=0):
        wurf = w20()

        if wurf > self.eigenschaften[glossar[probe]] + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {self.eigenschaften[glossar[probe]]} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0):
        """
        
        """

        ausgleich = self.fertigkeiten[probe][1]
        print(f"Starte Fertigkeitsprobe {probe} auf {self.fertigkeiten[probe][0]}, Ausgleich: {ausgleich}")

        for i, eigenschaft in enumerate(self.fertigkeiten[probe][0]):
            wurf = w20()
            print(f"\n{i+1}. Teilprobe auf {glossar[eigenschaft]}: Mit einem W20 wurde {wurf} geworfen!")

            ziel = self.eigenschaften[eigenschaft] + bias
            wurfdifferenz = ziel - wurf
            print(f"Ziel: {ziel}\nDifferenz: {wurfdifferenz}")

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
                print(f"neuer Ausgleich: {ausgleich}")
            
                if ausgleich < 0:
                    print(f"\nFertigkeitsprobe misslungen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {self.eigenschaften[eigenschaft]} mit Bias {bias}), Ausgleich wurde aufgebraucht ({ausgleich}).")

                    return None
                
                else:
                    print(f"Teilprobe fehlgeschlagen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {self.eigenschaften[eigenschaft]} mit Bias {bias}), verbleibender Ausgleich: {ausgleich}.")
            
            else:
                print(f"Teilprobe geglückt! {glossar[eigenschaft]} wurde mit {wurf} unterworfen (Ziel: {self.eigenschaften[eigenschaft]} mit Bias {bias}), verbleibender Ausgleich: ({ausgleich}).")
        
        print(f"\nFertigkeitsprobe geglückt! Übriger Ausgleich bei {ausgleich}.")

In [471]:
parnileo_parnilei = Einheit(data_parnileo_parnilei)
edgar_allan_parn = Einheit(data_edgar_allan_parn)

print(parnileo_parnilei.fertigkeiten["Tierkunde"])

[['KL', 'KL', 'IN'], 5]


In [472]:
print(f"Hallo, ich bin {parnileo_parnilei.name} und ich habe in Mut einen Wert von {parnileo_parnilei.eigenschaften['MU']}.")

parnileo_parnilei.eigenschaftsprobe("Mut")

Hallo, ich bin Parnileo Parnilei und ich habe in Mut einen Wert von 19.
Eigenschaftsprobe geschafft! Der nötige Mut-Wert von 19 wurde durch 15 unterworfen!


In [473]:
parnileo_parnilei.fertigkeitsprobe("Tierkunde")

Starte Fertigkeitsprobe Tierkunde auf ['KL', 'KL', 'IN'], Ausgleich: 5

1. Teilprobe auf Klugheit: Mit einem W20 wurde 14 geworfen!
Ziel: 13
Differenz: -1
neuer Ausgleich: 4
Teilprobe fehlgeschlagen! Klugheit wurde mit 14 nicht unterworfen (Ziel: 13 mit Bias 0), verbleibender Ausgleich: 4.

2. Teilprobe auf Klugheit: Mit einem W20 wurde 3 geworfen!
Ziel: 13
Differenz: 10
Teilprobe geglückt! Klugheit wurde mit 3 unterworfen (Ziel: 13 mit Bias 0), verbleibender Ausgleich: (4).

3. Teilprobe auf Intuition: Mit einem W20 wurde 8 geworfen!
Ziel: 5
Differenz: -3
neuer Ausgleich: 1
Teilprobe fehlgeschlagen! Intuition wurde mit 8 nicht unterworfen (Ziel: 5 mit Bias 0), verbleibender Ausgleich: 1.

Fertigkeitsprobe geglückt! Übriger Ausgleich bei 1.
